<a href="https://colab.research.google.com/github/sarisppp/PJ63_CE18/blob/master/ML_Stock_V.4_DecisionTree_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
import os, sys
import os.path
import numpy as np
import pandas as pd
import datetime
from pandas_datareader import data as pdr
from tqdm import tqdm, trange
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
pd.options.display.max_columns = 999
pd.options.display.max_rows = 999
from google.colab import drive
drive.mount('/content/drive')

from sklearn.preprocessing import MinMaxScaler


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [63]:
used_features = ['Timestamp','SET50.Close','EMAV(5)','Vol','EMAV(Vol,5)','MACD(13,5,5)','EMAV(MACD,5)','RSI(14)']
data = pd.read_excel("/content/drive/My Drive/Set50_20190314_20200820_1minute (1).xlsx",usecols =used_features)
data.info()
data.set_index("Timestamp",inplace=True)
data.head(20)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55920 entries, 0 to 55919
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Timestamp     55920 non-null  object 
 1   SET50.Close   55920 non-null  float64
 2   EMAV(5)       55916 non-null  float64
 3   Vol           55920 non-null  int64  
 4   RSI(14)       55907 non-null  float64
 5   MACD(13,5,5)  55908 non-null  float64
 6   EMAV(Vol,5)   55916 non-null  float64
 7   EMAV(MACD,5)  55904 non-null  float64
dtypes: float64(6), int64(1), object(1)
memory usage: 3.4+ MB


,SET50.Close,EMAV(5),Vol,RSI(14),"MACD(13,5,5)","EMAV(Vol,5)","EMAV(MACD,5)"
Timestamp,,,,,,,
13/03/19 14:26:00,1081.38,NaN,7698,NaN,NaN,NaN,NaN
13/03/19 14:27:00,1079.86,NaN,817,NaN,NaN,NaN,NaN
13/03/19 14:28:00,1081.50,NaN,2935,NaN,NaN,NaN,NaN
13/03/19 14:29:00,1081.32,NaN,94,NaN,NaN,NaN,NaN
13/03/19 14:30:00,1081.37,1081.0860,1900,NaN,NaN,2688.8894,NaN
13/03/19 14:31:00,1081.66,1081.2773,703,NaN,NaN,2026.9929,NaN
13/03/19 14:32:00,1081.42,1081.3249,3331,NaN,NaN,2461.7313,NaN
13/03/19 14:33:00,1080.90,1081.1833,6272,NaN,NaN,3731.9482,NaN
13/03/19 14:34:00,1081.01,1081.1255,1922,NaN,NaN,3128.7555,NaN


**Indicator**

In [64]:
def signal(data,periods=16):
  data['signal']=0
  for index,row in data.iterrows():
        #if index >= periods:
            if row['Vol'] > row['EMAV(Vol,5)'] :
              if row['SET50.Close']>row['EMAV(5)']:
                  
                  if row['MACD(13,5,5)']>row['EMAV(MACD,5)'] and row['RSI(14)']<70:
                    signal=1
                  else:
                    signal=0
              if row['SET50.Close']<row['EMAV(5)']:
               
                if row['MACD(13,5,5)']<row['EMAV(MACD,5)'] and row['RSI(14)']>30:
                  signal=-1
                else :
                  signal=0
            else:
              signal=0
            data.at[index, 'signal']= signal
    
     
  return data

In [65]:
data_new=data
data_new=signal(data_new)
data_new.head(50)

data=data.astype(float)
data=data.dropna()


========================================

In [66]:
data_new=data_new.dropna()
data_X = data_new.drop('signal',axis=1)
data_Y = data_new.signal

data_Y

Timestamp
13/03/19 14:42:00    0
13/03/19 14:43:00    0
13/03/19 14:44:00   -1
13/03/19 14:45:00    0
13/03/19 14:46:00    0
                    ..
20/08/20 16:42:00    0
20/08/20 16:43:00    0
20/08/20 16:44:00    0
20/08/20 16:45:00    0
20/08/20 16:46:00    0
Name: signal, Length: 55904, dtype: int64

In [67]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
train_X, test_X, train_y,test_y = train_test_split(data_X,data_Y,test_size=0.20, random_state=42)
print(train_X.info())
print("=====================================")
print(train_y.head())

DecisionTree = DecisionTreeClassifier()
DecisionTree.fit(train_X, train_y)

<class 'pandas.core.frame.DataFrame'>
Index: 44723 entries, 20/12/19 10:56:00 to 20/11/19 11:16:00
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   SET50.Close   44723 non-null  float64
 1   EMAV(5)       44723 non-null  float64
 2   Vol           44723 non-null  int64  
 3   RSI(14)       44723 non-null  float64
 4   MACD(13,5,5)  44723 non-null  float64
 5   EMAV(Vol,5)   44723 non-null  float64
 6   EMAV(MACD,5)  44723 non-null  float64
dtypes: float64(6), int64(1)
memory usage: 2.7+ MB
None
Timestamp
20/12/19 10:56:00      0
2020-01-07 10:49:00    0
20/07/20 11:55:00      0
20/07/20 10:50:00      0
17/01/20 10:06:00      0
Name: signal, dtype: int64


DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [68]:
#from sklearn import tree
#plt.figure()
#tree.plot_tree(DecisionTree)


In [69]:
y_predict=DecisionTree.predict(test_X)

from sklearn.metrics import classification_report 
report = classification_report(test_y,y_predict)
print(report)

              precision    recall  f1-score   support

          -1       0.76      0.76      0.76      1472
           0       0.94      0.94      0.94      8179
           1       0.78      0.77      0.78      1530

    accuracy                           0.89     11181
   macro avg       0.83      0.82      0.82     11181
weighted avg       0.89      0.89      0.89     11181



In [70]:
used_features = ['Timestamp','SET50.Close','EMAV(5)','Vol','EMAV(Vol,5)','MACD(13,5,5)','EMAV(MACD,5)','RSI(14)']
data = pd.read_excel("/content/drive/My Drive/Set50_20190314_20200820_1minute (1).xlsx",usecols =used_features)
data.set_index("Timestamp",inplace=True)
data.isnull().sum()
import time

#data['Timestamp'] = pd.to_datetime(data['Timestamp'], infer_datetime_format=True)
#data.info()
#data=data.astype(float)
data=data.dropna()
data_predict=DecisionTree.predict(data)
data=signal(data)
data['signal_pre']=data_predict


In [71]:
data.head(500)


,SET50.Close,EMAV(5),Vol,RSI(14),"MACD(13,5,5)","EMAV(Vol,5)","EMAV(MACD,5)",signal,signal_pre
Timestamp,,,,,,,,,
13/03/19 14:42:00,1081.53,1081.7645,1704,50.7389,0.1998,2618.7323,0.394,0,0
13/03/19 14:43:00,1081.18,1081.5697,740,48.2279,0.0599,1992.6252,0.283,0,0
13/03/19 14:44:00,1080.38,1081.1731,2758,42.9908,-0.1752,2247.8328,0.130,-1,-1
13/03/19 14:45:00,1080.90,1081.0821,719,47.0181,-0.2022,1738.1205,0.019,0,0
13/03/19 14:46:00,1082.00,1081.3881,1434,54.3627,0.0015,1636.7890,0.013,0,0
13/03/19 14:47:00,1081.33,1081.3687,632,49.8316,-0.0098,1301.7797,0.006,0,0
13/03/19 14:48:00,1081.22,1081.3191,320,49.1079,-0.0367,974.6458,-0.008,0,0
13/03/19 14:49:00,1081.46,1081.3661,536,50.7872,-0.0046,828.5502,-0.007,0,0
13/03/19 14:50:00,1081.84,1081.5241,569,53.4087,0.0863,741.8911,0.024,0,0
